In [1]:
import os

# This variable is used by helperbot to make the training deterministic
os.environ["SEED"] = "33223"

import logging
from pathlib import Path

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel

from helperbot import BaseBot, TriangularLR
from Utils import *

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
BERT_MODEL = 'bert-large-uncased'
CASED = False

In [3]:
df_train = pd.read_csv("gap-test.tsv", delimiter="\t")
df_val = pd.read_csv("gap-validation.tsv", delimiter="\t")
df_test = pd.read_csv("gap-development.tsv", delimiter="\t")

In [4]:
tokenizer = BertTokenizer.from_pretrained(
    BERT_MODEL,
    do_lower_case=CASED,
    never_split = ("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]", "[A]", "[B]", "[P]")
)
# These tokens are not actually used, so we can assign arbitrary values.
tokenizer.vocab["[A]"] = -1
tokenizer.vocab["[B]"] = -1
tokenizer.vocab["[P]"] = -1

In [5]:
model = GAPModel(BERT_MODEL, torch.device("cuda:0"))

Initing batchnorm
Initing linear
Initing batchnorm
Initing linear


In [6]:
model.load_state_dict(torch.load("./cache/model_cache/best.pth"))
model.eval()

GAPModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
   

In [7]:
df_inference = pd.read_csv("inference.tsv", delimiter="\t")
inference_ds = GAPDataset(df_inference, tokenizer)
inference_loader = DataLoader(
    inference_ds,
    collate_fn = collate_examples,
    batch_size=128,
    num_workers=2,
    pin_memory=True,
    shuffle=False
)

In [8]:
import tqdm
def predict(loader):
    model.eval()
    outputs = []
    with torch.set_grad_enabled(False):
        for *input_tensors, y_local in (loader):
            input_tensors = [x.to("cuda:0") for x in input_tensors]
            tmp = model(*input_tensors)
            outputs.append(tmp)
        outputs = torch.cat(outputs, dim=0)
    return outputs

In [9]:
preds = predict(inference_loader)
_, labels = torch.max(preds, 1)

In [10]:
labels.item()

0